In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义一个简单的CNN模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 卷积层 1: 输入是1个通道（MNIST图像是灰度的），输出是32个通道，卷积核大小为3x3
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # 卷积层 2: 输入是32个通道，输出是64个通道，卷积核大小为3x3
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # 最大池化层
        self.pool = nn.MaxPool2d(2, 2)
        # 全连接层，输入为7*7*64（图像从28x28缩小到7x7，通道数为64），输出为128
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        # 输出层，输入为128，输出为10（MNIST有10个类别）
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # 卷积 -> ReLU -> 池化
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 展平
        x = x.view(-1, 64 * 7 * 7)
        # 全连接层 -> ReLU
        x = F.relu(self.fc1(x))
        # 输出层（不加激活函数，因为交叉熵损失函数会处理 softmax）
        x = self.fc2(x)
        return x

# 定义超参数
batch_size = 64
learning_rate = 0.001
num_epochs = 5

# MNIST数据集的加载器
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型、损失函数和优化器
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        # 前向传播
        output = model(data)
        # 计算损失
        loss = criterion(output, target)
        # 反向传播
        loss.backward()
        # 更新权重
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试模型
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # 累加batch的损失
            pred = output.argmax(dim=1, keepdim=True)  # 获取最大概率的索引
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

# 运行训练和测试
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)



Using device: cuda
Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1147)>



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1147)>



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1147)>



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1147)>



100.0%


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324441
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.206574
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.226274
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.101152
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.040459
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.011668
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.021669
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.011565
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.137046
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.023550

Test set: Average loss: 0.0007, Accuracy: 9847/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.154384
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.025034
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.095330
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.028041
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.014213
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.096143
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.016039
Train Epoch: 2 [44800/6000